In [ ]:
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


# Extraction of needed datasets 

**Le but de la création de ce notebook est d'enrichir le graphe de connaissance par l'extraction des informations que je juge être un plus ceci permettra de caractériser mon projet par rapport au reste des travaux déjà existantes dans le cadre de Cybersecurity Knowledge Graphs** 🍀

Les informations que je compte ajouter sont : **APT Group**, **Techniques** utilisés par les attaquants, **tactics**, **Mitigations** pour contrer l'attaque . Tout type de malware appartient à un groupe apt spécifique et à partir de ce groupe on pourra avoir un ensemble d'informations sur les méthodes offensives du pirate informatique et les méthodes défensives pour le contrer. Malheureusement, cette information n'existe pas dans les rapports annotés (apt malwares notes) qui construit mon corpus :( . Pour cette raison, j'ai décidé d'opter pour une technique qui permettra d'attribuer à chaque malware le groupe APT auquel il appartient. 

In [ ]:
import requests
from lxml import etree
import time
import csv

In [ ]:
Tactics_info_list = []

Techniques_info_list = []
Techniques_url = []

Mitigations_info_list = []
Mitigations_info_list_temp = []
Mitigations_url = []

Groups_info_list = []
Groups_info_list_temp = []
Groups_url = []

In [ ]:
def get_tactics_info(url):
   
    html = requests.get(url)
    selector = etree.HTML(html.text)
   
    Tactics_ID = selector.xpath('//div[@class="card-body"]/div[1]/text()')
    Tactics_Created = selector.xpath('//div[@class="card-body"]/div[2]/text()')
    Tactics_LM = selector.xpath('//div[@class="card-body"]/div[3]/text()')
    Tactics_Name = selector.xpath('//div[@class="container-fluid"]/h1/text()')
    Tactics_Intros = selector.xpath('//div[@class="container-fluid"]/div[1]/div[1]/div[1]/p/text()')
    info_list = [Tactics_Name[0].strip(), Tactics_Intros[0], Tactics_ID[0].strip(), Tactics_Created[0], Tactics_LM[0]]
    Tactics_info_list.append(info_list)

  
    Techniques_url_infos = selector.xpath('//table[@class="table-techniques"]/tbody')
    for info in Techniques_url_infos:
        Techniques_ID = info.xpath('tr[@class="technique"]/td[1]/a/text()')
        for ID in Techniques_ID:
            Techniques_url.append('https://attack.mitre.org/techniques/{}/'.format(str(ID.strip())))

In [ ]:
def get_techniques_info(url):
    headers = {
        'User-Agent': 'user-agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    html = requests.get(url, headers=headers)
    selector = etree.HTML(html.text)
  
    Techniques_Name = selector.xpath('//div[@class="container-fluid"]/h1/text()')
    Techniques_ID = selector.xpath('//div[@class="card-body"]/div/div/span[contains(text(),"ID")]/../text()')
    Techniques_Platforms = selector.xpath('//div[@class="card-body"]/div/div/span[contains(text(),"Platforms")]/../text()')
    Techniques_Tactic = selector.xpath('//div[@class="card-body"]/div/div/span[contains(text(),"Tactic")]/../text()')


    Techniques_Sub_tec = selector.xpath(
        '//div[@class="card-body"]/div/div/span[contains(text(),"Sub-techniques")]/../text()')
    if Techniques_Sub_tec[0].strip() != 'No sub-techniques':
        Techniques_Sub_tec = selector.xpath(
            '//div[@class="card-body"]/div/span[contains(text(),"Sub-techniques")]/../a/text()')
        Techniques_Sub_tec = str(len(Techniques_Sub_tec)) + ' sub-techniques'
    else:
        Techniques_Sub_tec = Techniques_Sub_tec[0].strip()
    Techniques_PR = selector.xpath(
        '//div[@class="card-body"]/div/span[contains(text(),"Permissions Required")]/../text()')
    if Techniques_PR != []:
        Techniques_PR = Techniques_PR[0].strip()
    Techniques_DS = selector.xpath('//div[@class="card-body"]/div/span[contains(text(),"Data Sources")]/../text()')
    if Techniques_DS != []:
        Techniques_DS = Techniques_DS[0].strip()

    info_list = [Techniques_Name[0].strip(), Techniques_ID[0].strip(), Techniques_Sub_tec,
                 Techniques_Tactic[0].replace('\n', '').replace(' ', ''), Techniques_Platforms[0],
                 Techniques_DS, Techniques_PR]
    Techniques_info_list.append(info_list)

In [ ]:
def get_mitigations_url():
    headers = {
        'User-Agent': 'user-agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    html = requests.get('https://attack.mitre.org/mitigations/enterprise/', headers=headers)
    selector = etree.HTML(html.text)
 
    Mitigation_ID = selector.xpath('//div[@class="overflow-x-auto"]/table/tbody/tr/td[1]/a/text()')
    Mitigation_Name = selector.xpath('//div[@class="overflow-x-auto"]/table/tbody/tr/td[2]/a/text()')
    Mitigation_Des = selector.xpath('//div[@class="overflow-x-auto"]/table/tbody/tr/td[3]/text()')
    for i in range(0, len(Mitigation_Des)):
        Mitigations_url.append('https://attack.mitre.org/mitigations/{}/'.format(str(Mitigation_ID[i].strip())))
        info_list = [Mitigation_Name[i], Mitigation_ID[i], Mitigation_Des[i].strip()]
        Mitigations_info_list.append(info_list)


In [ ]:
def get_mitigations_info(url):
    headers = {
        'User-Agent': 'user-agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    html = requests.get(url, headers=headers)
    selector = etree.HTML(html.text)
 
    Tec_Addressed_by_Mitigation = selector.xpath(
        '//div[@class="container-fluid"]/table/tbody/tr[@class="technique"]/td[3]/a/text()')
    Mitigations_info_list_temp.append(Tec_Addressed_by_Mitigation)


In [ ]:
def get_groups_url():
    headers = {
        'User-Agent': 'user-agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    html = requests.get('https://attack.mitre.org/groups/', headers=headers)
    selector = etree.HTML(html.text)
   
    Associated_Groups = selector.xpath('//table[@class="table table-bordered table-alternate mt-2"]/tbody/tr/td[1]/text()')
    for group in Associated_Groups:
        Groups_info_list_temp.append(group.strip())
    Group_ID = selector.xpath('//table[@class="table table-bordered table-alternate mt-2"]/tbody/tr/td[2]/a/@href')
    for ID in Group_ID:
        Groups_url.append('https://attack.mitre.org{}/'.format(str(ID)))

In [ ]:
def get_groups_info(url):
    headers = {
        'User-Agent': 'user-agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
    html = requests.get(url, headers=headers)
    selector = etree.HTML(html.text)
 
    Tecs_Used_by_Group = selector.xpath(
        '//table[@class="table techniques-used table-bordered mt-2"]/tbody/tr/td[2]/a/text()')
    Group_Name =selector.xpath('//div[@class="container-fluid"]/h1/text()')
    Group_ID =selector.xpath('//div[@class="card-body"]/div[1]/text()')
    info_list = [Group_Name[0].strip(), Group_ID[0].replace(':', '').replace(' ', ''), Tecs_Used_by_Group]
    Groups_info_list.append(info_list)

In [ ]:
def get_urls1():
    urls = ['https://attack.mitre.org/tactics/TA000{}/'.format(str(i)) for i in range(1, 10)]
    urls.extend(['https://attack.mitre.org/tactics/TA0010/', 'https://attack.mitre.org/tactics/TA0011/',
                 'https://attack.mitre.org/tactics/TA0040/'])
    for url in urls:
        get_tactics_info(url)
        time.sleep(0.05)

In [ ]:
def get_urls2():
    for url in Techniques_url:
        get_techniques_info(url)
        time.sleep(0.05)

In [ ]:
def get_urls3():
    get_mitigations_url()
    for url in Mitigations_url:
        get_mitigations_info(url)
        time.sleep(0.05)

    for i in range(0, len(Mitigations_info_list_temp)):
        Mitigations_info_list[i].append(Mitigations_info_list_temp[i])

In [ ]:
def get_urls4():
    get_groups_url()
    for url in Groups_url:
        get_groups_info(url)
        time.sleep(0.05)

    for i in range(0, len(Groups_info_list)):
        Groups_info_list[i].append(Groups_info_list_temp[i])

In [ ]:
len(Groups_info_list_temp)

266

In [ ]:
len(Groups_info_list)

133

In [ ]:
get_urls1()
get_urls2()
get_urls3()
get_urls4()
    

**Datasets headers**

In [ ]:
header1 = ['Name', 'Intro', 'ID', 'Created', 'Last_Modified']
header2 = ['Name', 'ID', 'Sub-Tec', 'Tactic', 'Platforms', 'Data Sources', 'Permissions Required']
header3 = ['Name', 'ID', 'Description', 'Tecs Addressed by Mitigation']
header4 = ['Name', 'ID', 'Tecs Used by Group', 'Associated Groups']

**Create csv files**

In [ ]:
csvfile1 = open('ATT&CK MATRICES Tac.csv', 'w', errors='ignore', newline='')
csvfile2 = open('ATT&CK MATRICES Tec.csv', 'w', errors='ignore', newline='')
csvfile3 = open('ATT&CK MATRICES Miti.csv', 'w', errors='ignore', newline='')
csvfile4 = open('ATT&CK MATRICES Group.csv', 'w', errors='ignore', newline='')

**Filling csv files with extracted data**

In [ ]:
sheet1 = csv.writer(csvfile1)
sheet2 = csv.writer(csvfile2)
sheet3 = csv.writer(csvfile3)
sheet4 = csv.writer(csvfile4)

sheet1.writerow(header1)
sheet2.writerow(header2)
sheet3.writerow(header3)
sheet4.writerow(header4)
  
for list1 in Tactics_info_list:
    sheet1.writerow(list1)
for list2 in Techniques_info_list:
    sheet2.writerow(list2)
for list3 in Mitigations_info_list:
    sheet3.writerow(list3)
for list4 in Groups_info_list:
    sheet4.writerow(list4)

In [ ]:
csvfile1.close()
csvfile2.close()
csvfile3.close()
csvfile4.close()